In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
from PIL import Image
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torchvision
import numpy as np
from torch.utils.data import random_split
from torch.utils.data import Subset


In [ ]:
seed=42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
label_map = {
    'airplane': 0,
    'automobile': 1,
    'bird': 2,
    'cat': 3,
    'deer': 4,
    'dog': 5,
    'frog': 6,
    'horse': 7,
    'ship': 8,
    'truck': 9
}
class my_dataset(Dataset):
    def __init__(self,img_dir,csv_path,transform=None):
        self.img_dir=img_dir
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]
        self.labels=pd.read_csv(csv_path)#csv文件，用pandas处理
        self.transform = transform
    
    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self,idx):
        img_id=self.labels.iloc[idx,0]
        label_str=self.labels.iloc[idx,1]
        label=label_map[label_str]
        img_path = os.path.join(self.img_dir,f'{img_id}'+'.png')
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image,label




In [ ]:
img_dir = r"C:\Users\Lenovo\AI\project\data\train"
csv_path = r"C:\Users\Lenovo\AI\project\data\trainLabels.csv"
transform_for_stats=transforms.ToTensor()
dataset_for_stats=my_dataset(img_dir,csv_path,transform=transform_for_stats)
def get_mean_std(dataset, ratio=0.01):##取一小部分数据集来近似获得均值和标准差以用于归一化
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=int(len(dataset) * ratio), shuffle=True, num_workers=0
    )
    train = iter(dataloader).__next__()[0]
    mean = np.mean(train.numpy(), axis=(0, 2, 3))
    std = np.std(train.numpy(), axis=(0, 2, 3))
    return mean, std

data_mean,data_std=get_mean_std(dataset_for_stats)
transform_for_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=data_mean,
                         std=data_std),
])
transform_for_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=data_mean,
                         std=data_std),
])

In [ ]:
dataset1 = my_dataset(img_dir, csv_path, transform=transform_for_train)
dataset2 = my_dataset(img_dir, csv_path, transform=transform_for_test)
train_size=int(0.9*len(dataset1))
val_size=len(dataset1)-train_size
# 加种子确保可复现
indices = list(range(len(dataset1)))
train_indices, val_indices = random_split(indices, [train_size, val_size], generator=torch.Generator().manual_seed(seed))
#构造子集（注意：不同 transform 绑定的是不同 dataset）
trainset = Subset(dataset1, train_indices)
valset   = Subset(dataset2, val_indices)
train_dataloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=0)
val_dataloader   = DataLoader(valset,   batch_size=50, shuffle=False, num_workers=0)


In [ ]:

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B, 32, 32, 32]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                          # [B, 32, 16, 16]
            nn.Conv2d(32, 64, kernel_size=3, padding=1), # [B, 64, 16, 16]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                          # [B, 64, 8, 8]
            nn.Flatten(),                                # [B, 64*8*8]
            nn.Linear(64*8*8, 128),
            nn.ReLU(),
            nn.Linear(128, 10)  # 输出类别数为 10
        )
    
    def forward(self, x):
        return self.net(x)
    
model=SimpleCNN()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def plot_training_progress(losses, accuracies):
    epochs = range(1, len(losses) + 1) 
    plt.figure(figsize=(12,5))
    # 绘制Loss曲线
    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, 'r-', label='Train Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    # 绘制Accuracy曲线
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracies, 'b-', label='Train Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy')
    plt.legend()
    plt.show()

In [ ]:
def train(model,train_loader,criterion,optimizer,device):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        ##参数更新
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ##累计正确预测个数与总数
        running_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    return running_loss/total,correct/total

In [ ]:
def evaluate(model, data_loader, criterion, device):
    model.eval()  # 进入评估模式
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in data_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * imgs.size(0)  # 加总整个 batch 的 loss

            _, preds = torch.max(outputs, 1)  # 获得预测类别
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return total_loss/total,correct/total



In [ ]:
num_epochs = 60
train_losses=[]
train_accuracies=[]
val_losses=[]
val_accuracies=[]
for epoch in range(num_epochs):
    train_avg_loss,train_acc=train(model,train_dataloader,criterion,optimizer,device)
    val_avg_loss,val_acc=evaluate(model,val_dataloader,criterion,device)
    ##计算每个epoch后训练集和验证集的平均loss和accuracy
    train_losses.append(train_avg_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_avg_loss)
    val_accuracies.append(val_acc)
    print(f"Epoch [{epoch+1}/{num_epochs}], TrainLoss: {train_avg_loss:.4f}, TrainAccuracy: {train_acc*100:.2f}%")
    print(f"Epoch [{epoch+1}/{num_epochs}], ValLoss: {val_avg_loss:.4f}, ValAccuracy: {val_acc*100:.2f}%")
plot_training_progress(train_losses, train_accuracies)
plot_training_progress(val_losses, val_accuracies)

In [ ]:
torch.save(model.state_dict(), "CNN_model_01.pth")


In [ ]:
test_dataset=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform_for_train)
test_dataloader=torch.utils.data.DataLoader(test_dataset,batch_size=50,shuffle=False,num_workers=0)
model = SimpleCNN()
model.load_state_dict(torch.load(r"CNN_model_01.pth"))
model.eval()

correct=0
total=0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
with torch.no_grad():
    for imgs,labels in val_dataloader:
        imgs,labels = imgs.to(device),labels.to(device)
        outputs = model(imgs)
        _,predicted = torch.max(outputs,1)
        total +=labels.size(0)
        correct +=(predicted==labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

